In [ ]:
!pip install langdetect -q
!pip install newspaper3k -q

In [ ]:
import newspaper
import pandas as pd
import requests
from langdetect import detect
import time
from newspaper.article import Article, ArticleException

In [ ]:
def scrape_article(url, target_date, max_retries=3):
    retry_count = 0
    article_data = []

    while retry_count < max_retries:
        try:
            article = Article(url, language='en')
            article.download()
            article.parse()

            if len(article.text) < 10:
                print(f"Skipping article with insufficient text: {article.url}")
                break

            article_language = detect(article.text)
            if article_language == 'en':
                if (
                    article.publish_date
                    and article.publish_date.date() > pd.to_datetime(target_date).date()
                ):
                    article_data.append({
                        'title': article.title,
                        'body': article.text,
                        'author': article.authors if article.authors else "No author found",
                        'publish_date': article.publish_date.strftime('%Y-%m-%d')
                    })
                else:
                    print(f"Skipping older article: {article.url}")
                break
            else:
                print(f"Skipping non-English article: {article.url}")
                break
        except ArticleException as ae:
            print(f"Failed to process article: {str(ae)}")
            retry_count += 1
            time.sleep(2)  # Wait for a short time before retrying
            continue
        except Exception as e:
            print(f"Unhandled exception: {str(e)}")
            break

    if not article_data:
        print("No article found matching the criteria.")
        return None

    return article_data




if result is not None:
    st.write(result)
else:
    st.write("No data to display.")

In [ ]:
start_time = time.time()
target_date = '2022-01-16'
result = scrape_article(link, target_date)#replace link here with link to scrape
if result is not None:
    st.write(result)
else:
    st.write("No data to display.")
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")